# Set up

In [1]:
import jsonlines
import os
import pandas as pd
import copy
import json
import re

# Rename agriculture file name

(no need to run now)

In [2]:
# root_path = os.path.abspath(os.path.dirname(os.getcwd()))
# data_path = os.path.join(root_path, 'data')
# inds_path = os.path.join(data_path, 'agriculture')
# fina_path = os.path.join(inds_path, 'Financial Statistic_Agriculture')
# for i in os.listdir(fina_path):
#     comp_code = i.split('_')[0]
#     old_name = fina_path + os.sep + i
#     new_name = fina_path + os.sep + comp_code + "_financial_ratios.csv"
#     os.rename(old_name, new_name) 

# Assemble data

Aim is to structure a company like:
```
{"CTVA": {
"name": "Corteva, Inc.", 
"location": ["Chestnut Run Plaza 735", "PO Box 80735", "United States"], 
"website": "http://www.corteva.com", 
"section": "Basic Materials", 
"industry": "Agricultural Inputs", 
"profile": "https://finance.yahoo.com/quote/CTVA/profile?p=CTVA"},
"stock prices": pandas.framework,
"ROA": pandas.framework,
"state": NY
}
```

One `json` file corresponds to a company.  

However, `json` can't load the `pandas.framework` directly, and to avoid losing data and adding escape characters, we use `pandas.to_dict()` to encode `pandas.framework`, then we can store it in `json`.

As for decoding the `json`, we use `pandas.DataFrame.from_dict()` to transform the `dict` to `dataframe`. Thus, the final structure for each file (company) is:
```
{"CTVA": {
"name": "Corteva, Inc.", 
"location": ["Chestnut Run Plaza 735", "PO Box 80735", "United States"], 
"website": "http://www.corteva.com", 
"section": "Basic Materials", 
"industry": "Agricultural Inputs", 
"profile": "https://finance.yahoo.com/quote/CTVA/profile?p=CTVA"},
"stock prices": dict,
"ROA": dict,
"state": NY
}
```

## Get the company basic information

In [3]:
root_path = os.path.abspath(os.path.dirname(os.getcwd()))
web_path = os.path.join(root_path, 'webpage')

total = {}
for f in os.listdir(web_path):
    if f.endswith('.jsonl'):
        inds = f.replace('.jsonl', '')
        total[inds] = {}
#         total[inds] = []
        file_path = os.path.join(web_path, f)
        with jsonlines.open(file_path, 'r') as lines:
            for line in lines:
                total[inds][list(line.keys())[0]] = list(line.values())[0]

In [4]:
total.keys()

dict_keys(['agriculture', 'energy', 'travel'])

In [5]:
agriculture = total['agriculture']
energy = total['energy']
travel = total['travel']

## Store the stock prices

In [6]:
data_path = os.path.join(root_path, 'data')
cp = copy.deepcopy(total)

for inds in cp.keys():
    inds_path = os.path.join(data_path, inds)
    for comp in cp[inds].keys():
        file_name = comp + '.csv'
        file_path = os.path.join(inds_path, file_name)
        try:
            total[inds][comp]["stock prices"] = pd.read_csv(file_path).to_dict()
        except:
            del total[inds][comp]

## Store the CFP data

In [7]:
for inds in total.keys():
#     print(inds)
    inds_path = os.path.join(data_path, inds)
    fina_path = os.path.join(inds_path, 'Financial_ratios')

    cp = copy.deepcopy(total)

    cfp = ['ROA1', 'ROA2', 'Operating Margin', 'Net Margin', 'Return On Equity', 
           'Current Ratio', 'Quick ratio', 'ROC', 'D/E ratio', 'EPS']

    for comp in cp[inds].keys():
        try:
            file_name = comp + '_financial_ratios.csv'
            file_path = os.path.join(fina_path, file_name)
            df = pd.read_csv(file_path)
            for i in cfp:
                value = df.loc[df['name'] == i]
                total[inds][comp][i] = value.to_dict()
        except:
            del total[inds][comp]

## Store the State data of US

REF: https://gist.github.com/rogerallen/1583593

In [8]:
# https://www.corteva.com/contact-us.html
agriculture['CTVA']['location'] = ['Chestnut Run Plaza 735', 'PO Box 80735', 
                                   'Wilmington, DE 19805-0735', 'United States']
# https://ir.mgpingredients.com/governance/governance-contacts
agriculture['MGPI']['location'] = ['Cray Business Plaza', '100 Commercial Street PO Box 130',
                                   '100 Commercial Street, P.O. Box 130 ', 'Atchison, KS 66002',
                                   'United States']

In [9]:
# load us state abbre data
file_name = os.path.join(data_path, 'us_state.json')
with open(file_name, 'r') as f:
    us_state_dict = json.load(f)

count = 0
for inds in total.keys():
#     print('\n',inds)
    for comp, v in total[inds].items():
        try:
            if v['location'][-1] == 'United States':
                state_str = v['location'][-2]
    #             print(comp, v['location'])
                state_pattern = re.compile(r'[A-Z]{2}(?= \d)')
                try:
                    state = re.findall(state_pattern, state_str)[0]
#                     print(comp, v['location'], state)
                    state = us_state_dict[state]
                    total[inds][comp]['state'] = state
                    count += 1
                except:
                    total[inds][comp]['state'] = None
                    pass
            else:
                total[inds][comp]['state'] = None
        except:
            total[inds][comp]['state'] = None
        count

In [21]:
for inds in total.keys():
    print(inds)
    for comp, v in total[inds].items():
        if total[inds][comp]['state']:
            print(comp, total[inds][comp]['state'])
    print('\n')

agriculture
CTVA Delaware
MBII California
AVD California
IPI Colorado
MGPI Kansas
SMG Ohio
CF Illinois
YTEN Massachusetts
CTA-PB Delaware
UAN Texas
CTA-PA Delaware
RKDA California
FMC Pennsylvania
MOS Florida


energy
PTEN Texas
ICD Texas
HP Oklahoma


travel
TNL Florida
NCLH Florida
MKGI Florida
LIND New York
TZOO New York
EXPE Washington
TRIP Massachusetts




In [17]:
total['agriculture']['CTVA']['location']

['Chestnut Run Plaza 735',
 'PO Box 80735',
 'Wilmington, DE 19805-0735',
 'United States']

In [18]:
total['agriculture']['CTVA']['state']

'Delaware'

In [9]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="find state")
location = geolocator.geocode("Cray Business Plaza")
print(location.address)

Cray, Powys, Cymru / Wales, United Kingdom


# Store the data into json for each company

In [13]:
for inds in total.keys():
#     print(inds)
    inds_path = os.path.join(data_path, inds)

    for comp in total[inds].keydous():
        file_name = comp + '.json'
        file_path = os.path.join(inds_path, file_name)
        with open(file_path, 'w') as file:
            json.dump({comp:total[inds][comp]}, file)

# Read the json

In [11]:
t = []
for comp in total['agriculture'].keys():
    file_name = comp + '.json'
    file_path = os.path.join(inds_path, file_name)
    with open(file_path, 'r') as file:
        t.append(json.load(file))

In [12]:
# pd.DataFrame.from_dict(t[0]['CTVA']['stock prices'])